In [ ]:
pip install langchain==0.1.0

In [ ]:
pip install langchain-community==0.0.10 langchain-openai==0.0.2 langchain-pinecone==0.0.1 sec-edgar-downloader==5.0.0 pypdf==3.17.4 tiktoken==0.5.2


In [ ]:
pip install python-dotenv==1.0.0 pinecone-client==3.0.0

In [ ]:
pip install langchain_community.document_loaders

In [ ]:
pip install langchain_text_splitters

In [ ]:
pip install -U langchain langchain-community langchain-openai langchain-pinecone langchain-text-splitters sec-edgar-downloader pypdf tiktoken python-dotenv pinecone-client

In [ ]:
import os
import glob
from dotenv import load_dotenv
from sec_edgar_downloader import Downloader
from langchain_community.document_loaders import PyPDFLoader, BSHTMLLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()

def fetch_10k_filings(ticker, email="maturamufaro@gmail.com"):
    print(f"📥 Fetching 10-K for {ticker}...")
    
    # Check where we are currently running
    current_dir = os.getcwd()
    print(f"📂 Current Working Directory: {current_dir}")
    
    dl = Downloader("MyRAGPortfolio", email)
    dl.get("10-K", ticker, limit=1)

    # 1. Define the base folder based on your specific path
    # We look for: current_dir/sec-edgar-filings/AAPL/10-K/
    base_search_path = os.path.join(current_dir, "sec-edgar-filings", ticker, "10-K")
    
    if not os.path.exists(base_search_path):
        print(f"❌ Error: The folder {base_search_path} does not exist.")
        return None

    # 2. Find the Accession Number folder (e.g., 0000320193-25-000079)
    # We take the first folder we find inside '10-K'
    subfolders = [f.path for f in os.scandir(base_search_path) if f.is_dir()]
    
    if not subfolders:
        print(f"❌ No filing folders found inside {base_search_path}")
        return None
    
    target_folder = subfolders[0]
    print(f"📂 Found Accession Folder: {target_folder}")

    # 3. Look for the document inside that folder
    # Priority: primary-document.html -> primary-document.pdf -> *.txt
    possible_files = [
        os.path.join(target_folder, "primary-document.html"),
        os.path.join(target_folder, "primary-document.htm"),
        os.path.join(target_folder, "primary-document.pdf"),
        os.path.join(target_folder, "full-submission.txt")
    ]
    
    file_path = None
    for p in possible_files:
        if os.path.exists(p):
            file_path = p
            break
            
    # Fallback: Just grab the first file in the directory if standard names fail
    if not file_path:
        all_files = os.listdir(target_folder)
        if all_files:
            file_path = os.path.join(target_folder, all_files[0])

    if file_path:
        print(f"✅ Found filing file: {file_path}")
    else:
        print(f"❌ Folder exists but contains no files: {target_folder}")
        
    return file_path

def load_document(file_path):
    print("⏳ Loading document... (this might take a moment for large HTML files)")
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".html") or file_path.endswith(".htm"):
        loader = BSHTMLLoader(file_path)
    else:
        loader = TextLoader(file_path)
    return loader.load()

def clean_text(text):
    import re
    # Remove "Page X of Y" artifacts
    text = re.sub(r"Page \d+ of \d+", "", text)
    text = re.sub(r"Table of Contents", "", text)
    # Collapse multiple newlines
    text = re.sub(r"\n\s*\n", "\n\n", text)
    return text

def ingest_data():
    file_path = fetch_10k_filings("AAPL", "maturamufaro@gmail.com")
    
    if not file_path:
        return

    # LOAD
    try:
        raw_documents = load_document(file_path)
        print(f"   Loaded content (approx {len(raw_documents)} pages/sections).")
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return

    # CHUNK
    print("✂️  Chunking text...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )

    documents = text_splitter.split_documents(raw_documents)
    
    # CLEAN & METADATA
    print("🧹 Cleaning text...")
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["company"] = "AAPL"
        doc.metadata["year"] = "2023"

    print(f"🧩 Created {len(documents)} chunks.")
    
    # EMBED & STORE
    if os.getenv("OPENAI_API_KEY") and os.getenv("PINECONE_API_KEY"):
        print("🚀 Upserting to Pinecone (this takes about 30-60s)...")
        index_name = "financial-10k"
        
        batch_size = 100
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i+batch_size]
            PineconeVectorStore.from_documents(
                batch,
                OpenAIEmbeddings(),
                index_name=index_name
            )
            print(f"   Uploaded batch {i // batch_size + 1}")
        print("✅ Ingestion Complete!")
    else:
        print("🛑 API Keys missing. Skipping Pinecone upload.")

if __name__ == "__main__":
    ingest_data()

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# 1. Load Secrets
load_dotenv(override=True)

def start_chat():
    print("🧠 Initializing the Financial Analyst AI...")

    # 2. Connect to the "Brain" (LLM)
    # We use gpt-4o-mini because it's fast, cheap, and smart enough for this.
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 3. Connect to the (Vector DB)
    embeddings = OpenAIEmbeddings()
    vectorstore = PineconeVectorStore(
        index_name="financial-10k", 
        embedding=embeddings
    )
    
    # 4. Set up the Retriever
    # "k=3" means: "Find the 3 most relevant chunks for every question"
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # 5. Create the System Prompt (The Persona)
    # This instructs the AI to ONLY use the retrieved data.
    system_prompt = (
        "You are a senior financial analyst. "
        "Use the provided context to answer the user's question. "
        "If the answer is not in the context, say 'I could not find information relating to that, sorry'. "
        "Keep your answers concise and professional.\n\n"
        "Context: {context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])

    # 6. Build the Chain
    # This connects: Retriever -> Document Combiner -> LLM
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    print("✅ Ready! Ask a question about Apple (AAPL). Type 'exit' to quit.\n")

    # 7. Chat Loop
    while True:
        query = input("User: ")
        if query.lower() in ["exit", "quit"]:
            break

        # Run the RAG pipeline
        response = rag_chain.invoke({"input": query})
        
        print(f"\nAI: {response['answer']}")
        
        # Optional: Show sources (Proof it's working)
        print("\n--- Source Documents ---")
        for i, doc in enumerate(response["context"]):
            print(f"Source {i+1}: {doc.page_content[:100]}...") # Print first 100 chars
        print("-" * 30 + "\n")

if __name__ == "__main__":
    start_chat()